# Make event files for probe task

In [1]:
import os
import re
from os.path import join as opj
from glob import glob
import scipy.io as sio
import numpy as np
import pandas as pd

Helper functions:

In [2]:
str_extract = lambda pattern, s: re.search(pattern, s).group(0)
num_extract = lambda pattern, s: int(str_extract(pattern, s))

Experiment directories:

In [3]:
# Project directory
scratch_dir = os.environ['PI_SCRATCH']
project_dir = opj(scratch_dir, 'OPUS')

# Raw behavioral files
in_dir = opj(project_dir, 'session_data')
in_files = glob(opj(in_dir, '*probe*'))
in_files.sort()

# Output directory
out_dir = opj(project_dir, 'BIDS_data')
out_template = opj(out_dir, 'sub-%02d', 'func', 'sub-%02d_task-probe_run-%02d_events.tsv')
print(out_template % (3, 3, 1))

/scratch/groups/hyo/OPUS/BIDS_data/sub-03/func/sub-03_task-probe_run-01_events.tsv


MAIN LOOP: Read raw behavioral files and save as event files

In [4]:
for f in in_files:
    tmp_in = sio.loadmat(f)
    tmp_data = tmp_in['Data']['trial'][0][0][0]

    # Get output file from filename
    tmp_sub = num_extract('(?<=sll\_opusfmri\_)[0-9]+', f)
    tmp_run = num_extract('(?<=probe\.)[0-9]+', f)
    tmp_outf = out_template % (tmp_sub, tmp_sub, tmp_run)

    # Trial types + timing
    tmp_condition = [c[0].lower() for c in tmp_data['condition']]
    tmp_onset = [o[0][0] for o in tmp_data['ons']]
    tmp_duration = [d[0][0] for d in tmp_data['dur']]
    tmp_rt = [t[0][0] for t in tmp_data['rt']]

    # Stimuli
    tmp_stim = [s[0].replace('.png', '') for s in tmp_data['stim']]

    # Ratings (tmp_scale == -1: reverse coded)
    tmp_scale = tmp_in['Data']['scale'][0][0][0][0]
    tmp_ratings_raw = [r[0][0] for r in tmp_data['response']]
    tmp_ratings = np.array(tmp_ratings_raw)
    tmp_ratings = tmp_ratings if (tmp_scale == 1) else 5-tmp_ratings

    # Assemble data frame
    tmp_out = pd.DataFrame(list(zip(tmp_onset, tmp_duration,
                                    tmp_condition, tmp_rt,
                                    tmp_stim, tmp_ratings)),
                          columns = ['onset', 'duration', 'trial_type', 'response_time', 'stimulus', 'rating'])

    tmp_out.to_csv(tmp_outf, sep='\t', na_rep='n/a', index=False)